# Projet 4

In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import requests


Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [2]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    
    r = requests.get(url)
    content = r.content.decode('utf-8')
    soup = BeautifulSoup(content) 
    div_tag = soup.find_all('div', attrs = {'class' : 'pricing-table'})
    
    rows = {}
    for div in div_tag : 
        row = {}
        row["price"] = str(div.find('span', attrs = {'class' : 'pricing-table-price'}).contents[0]).strip()
        row["storage"] = div.find('ul', attrs = {'class' : 'pricing-table-list'}).find_all('li')[3].text.split()[0]
        row["databases"] = int(div.find('ul', attrs = {'class' : 'pricing-table-list'}).find_all('li')[4].text.split()[0])
        plan = div.find('h2').text
        rows[plan] = row
    
    prices = {}
    return rows

res = pd.DataFrame(get_prices_from_url(URL_PAGE3))
res

,Personal,Small Business,Enterprise,Privilege
price,$5,$25,$45,$99
storage,1GB,10GB,100GB,1TB
databases,1,5,25,100


Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [3]:
url_melusine = r'https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33'
r = requests.get(url_melusine)
content = r.content.decode('utf-8')
soup = BeautifulSoup(content)

soup.find('div' , attrs = {'class' : 'product-detail-info-title'}).find("h1").text


'Mélusine Bio'

In [4]:
def extract_beer_infos(url):
    infos = {
        'name': None,
        'note': None,
        'price': None,
        'volume': None,
    }
    r = requests.get(url)
    content = r.content.decode('utf-8')
    soup = BeautifulSoup(content)
    infos['name'] = soup.find('div' , attrs = {'class' : 'product-detail-info-title'}).find("h1").text
    infos['note'] = float(soup.find('span' , attrs = {'class' : 'label-stars'}).text.replace('(','').replace(')','').replace(',','.'))
    infos['price'] = float(soup.find('span' , attrs = {'class' : 'price'}).text.replace(',','.').split()[0])
    infos['volume'] = int(soup.find('dd' , attrs = {'class' : 'small-6'}).find_next_sibling("dd").text.split()[0])
    print('done')
    return infos


url_melusine = r'https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33'
extract_beer_infos(url_melusine)
# product-detail-info-title

done


{'name': 'Mélusine Bio', 'note': 3.29, 'price': 38.99, 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [5]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [6]:
r = requests.get(URL_BEERLIST_FRANCE)
r.json()['items'][0]

{'title': 'Gallia East IPA',
 'description': '',
 'images': [{'image': '/globalassets/gallia-east-ipabottle.33_15073_0.png',
   'badge': {'url': '/', 'alt': None, 'title': None},
   'name': None}],
 'contentReference': '/fr-fr/p/bieres/gallia-east-ipa-bottle-.33',
 'redirectedUrl': None,
 'details': 'IPA | 6,0% | 33 cl',
 'worksWithLabel': None,
 'productIcon': None,
 'searchItemType': 0,
 'promotedPages': [],
 'code': 'Gallia-East-IPABottle.33_15073',
 'inStock': True,
 'isComingSoon': False,
 'isBottleOrCan': True,
 'inReserveStock': True,
 'style': 'IPA',
 'alcoholPercentage': 6.0,
 'volume': 33.0,
 'displayInformationPrice': {'currency': '€',
  'price': '3,09 €',
  'filterPrice': 3.09,
  'fromPrice': None,
  'discountPrice': None,
  'discountPercentage': '0',
  'valueIndicator': None,
  'pricePerLiter': '9,36 €',
  'depositMoney': None,
  'displayDepositMoneyAndPricePerLiter': False,
  'hasDiscount': False,
  'vatPercentage': '20%'},
 'amountInCart': 0,
 'isInWishlist': False,
 'co

In [7]:
def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    r = requests.get(URL_BEERLIST_FRANCE)
    for i, page in enumerate(r.json()['items']):
        beer_pages.append('https://www.beerwulf.com'+page['contentReference'])
    
    # Sequential version (slow):
    beers = []
    for page in beer_pages:
        beers.append(extract_beer_infos(page))
    # Parallel version (faster):
    # beers = []
    return beers

extract_beer_list_infos(URL_BEERLIST_FRANCE)

done
done
done
done
done
done
done
done
done
done
done


[{'name': 'Gallia East IPA', 'note': 3.8, 'price': 42.99, 'volume': 33},
 {'name': 'La Lager Sans Gluten de Vézelay',
  'note': 2.78,
  'price': 38.99,
  'volume': 25},
 {'name': 'Brasserie De Sutter Brin de Folie',
  'note': 3.42,
  'price': 44.99,
  'volume': 33},
 {'name': 'La Cristal IPA du Mont Blanc',
  'note': 3.44,
  'price': 44.99,
  'volume': 33},
 {'name': 'Mélusine Bio', 'note': 3.29, 'price': 38.99, 'volume': 33},
 {'name': 'La Parisienne Le Titi Parisien',
  'note': 3.5,
  'price': 38.99,
  'volume': 33},
 {'name': 'Gallia Session IPA', 'note': 3.61, 'price': 42.99, 'volume': 33},
 {'name': 'Ninkasi Brut IPA', 'note': 3.36, 'price': 44.99, 'volume': 33},
 {'name': 'Pietra', 'note': 3.17, 'price': 38.99, 'volume': 33},
 {'name': 'Desperados', 'note': 3.15, 'price': 35.99, 'volume': 33},
 {'name': 'Gallia West IPA', 'note': 3.56, 'price': 42.99, 'volume': 33}]

In [8]:
from multiprocessing import Pool

In [ ]:
# def extract_beer_list_infos2(url):
#     # Collecter les pages de bières à partir du JSON
#     beer_pages = []
#     r = requests.get(URL_BEERLIST_FRANCE)
#     for i, page in enumerate(r.json()['items']):
#         beer_pages.append('https://www.beerwulf.com'+page['contentReference'])
    
# #     p = Pool(4)
    
# #     res = p.map(extract_beer_infos, beer_pages)
    
#     return beer_pages

# beer_pages = extract_beer_list_infos(URL_BEERLIST_FRANCE)

In [ ]:
# beers = []

# with Pool() as pool:
#     beers = pool.map(extract_beer_infos, beer_pages)

In [124]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [125]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 3.298s

OK
